# Digit recognizer

<img src="digit-recognizer.png" height="400" width="400">

El objetivo de esta competencia es tomar una imagen de un digito escrito a mano, y determinar cual es el digito.

Los datos para esta competencia fueron tomados del dataset MNIST. El dataset MNIST es un clásico en la comunidad de aprendizaje automático que ha sido extensivamente estudiado. Más detalles sobre el dataset, incluyendo los algoritmos de aprendizaje automático que han sido probados y sus distintos niveles de éxito, pueden encontrarse en http://yann.lecun.com/exdb/mnist/index.html.

## Resolución con red neuronal multicapa

#### Instalación de paquetes necesarios

In [5]:
list.of.packages <- c('mxnet')

new.packages <- list.of.packages[!(list.of.packages %in% installed.packages()[,"Package"])]
if(length(new.packages)) install.packages(new.packages, repos = "https://cran.r-project.org")

library(mxnet)

## Carga de datos

In [6]:
train <- read.csv('data/train.csv', header=TRUE)
test <- read.csv('data/test.csv', header=TRUE)
train <- data.matrix(train)
test <- data.matrix(test)

train.x <- train[,-1]
train.y <- train[,1]

Cada imagen se representa como una fila en el dataset de entramiento/prueba. La escala de grises de cada imagen se representa con un entero entre [0, 255].

Para trabajar con redes neuronales es conveniente primero escalar estos valores.

In [7]:
train.x <- t(train.x/255)
test <- t(test/255)

Debemos transponer la matrix de entrada a npixeles x nejemplos, que es el formato de columnas aceptado por MXNet (y la convención de R).

Teniendo en cuenta la clase, el numero de digitos está distribuido bastante uniforme.

In [8]:
table(train.y)

train.y
   0    1    2    3    4    5    6    7    8    9 
4132 4684 4177 4351 4072 3795 4137 4401 4063 4188 

## Configuración de la red

In [9]:
data <- mx.symbol.Variable("data")
fc1 <- mx.symbol.FullyConnected(data, name="fc1", num_hidden=128)
act1 <- mx.symbol.Activation(fc1, name="relu1", act_type="relu")
fc2 <- mx.symbol.FullyConnected(act1, name="fc2", num_hidden=64)
act2 <- mx.symbol.Activation(fc2, name="relu2", act_type="relu")
fc3 <- mx.symbol.FullyConnected(act2, name="fc3", num_hidden=10)
softmax <- mx.symbol.SoftmaxOutput(fc3, name="sm")

1. En mxnet, utilizamos el símbolo de tipo de datos para configurar la red. data <- mx.symbol.Variable("data") utiliza "data" para representar los datos de entrada, por ejemplo, la capa de entrada.
2. Configuramos la primer capa oculta fc1 <- mx.symbol.FullyConnected(data, name="fc1", num_hidden=128). Esta capa tiene "data" como entrada, su nombre, y la cantidad de neuronas ocultas.
3. La activación se configura con act1 <- mx.symbol.Activation(fc1, name="relu1", act_type="relu"). La función de activación toma la salida de la primer capa fc1.
4. La segunda capa oculta toma los resultados de act1 como entradas, con su nombre "fc2" y el numero de neuronas ocultas: 64.
5. La segunda activación es casi lo mismo que act1, excepto que tenemos una fuente de entrada y nombre diferentes.
6. Esto genera la capa de salida. Porque solo hay 10 digitos, configuramos el numero de neuronas a 10.
7. Finalmente, configuramos la activación de softmax para obtener la predicción probabilística.

## Entrenamiento de la red
Antes de entrenar la red configuramos el dispositivo que vamos a usar:

In [10]:
devices <- mx.cpu()

In [11]:
mx.set.seed(0)
model <- mx.model.FeedForward.create(softmax, X=train.x, y=train.y,
                                        ctx=devices, num.round=10, array.batch.size=100,
                                        learning.rate=0.07, momentum=0.9,  eval.metric=mx.metric.accuracy,
                                        initializer=mx.init.uniform(0.07),
                                        epoch.end.callback=mx.callback.log.train.metric(100))

Warning message in mx.model.select.layout.train(X, y):
"Auto detect layout input matrix, use colmajor..
"

Start training with 1 devices
[1] Train-accuracy=0.859832935560859
[2] Train-accuracy=0.957666666666668
[3] Train-accuracy=0.971023809523813
[4] Train-accuracy=0.977714285714289
[5] Train-accuracy=0.981571428571432
[6] Train-accuracy=0.986309523809527
[7] Train-accuracy=0.988952380952383
[8] Train-accuracy=0.990880952380956
[9] Train-accuracy=0.992142857142861
[10] Train-accuracy=0.991095238095241


## Predicciones

In [13]:
preds <- predict(model, test)
dim(preds)

Warning message in mx.model.select.layout.predict(X, model):
"Auto detect layout input matrix, use colmajor..
"

[1]    10 28000

Preds es una matriz con 28000 filas y 10 columnas, y contiene las probabilidades de clasificación deseada de la capa de salida. Para extraer el valor máximo de cada fila usamos max.col:

In [14]:
pred.label <- max.col(t(preds)) - 1
table(pred.label)

pred.label
   0    1    2    3    4    5    6    7    8    9 
2816 3216 2753 2791 2709 2544 2762 2836 2780 2793 

## Submit

In [15]:
submission <- data.frame(ImageId=1:ncol(test), Label=pred.label)
write.csv(submission, file='submission.csv', row.names=FALSE,  quote=FALSE)

    El puntaje obtenido al momento del envío fue de 0.97014.

Fuente: http://mxnet.io/tutorials/r/mnistCompetition.html